In [1]:
# Import library-library
import os
import glob
import string

# Data Preparation and Preprocessing
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Word Embedding
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse

from sklearn.metrics.pairwise import cosine_similarity

# Input and Expansion Query
from textblob import TextBlob
from nltk.tokenize import wordpunct_tokenize
#from googletrans import Translator

#Split to train and test
from sklearn.model_selection import train_test_split

import mysql.connector
from mysql.connector import errorcode

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\RayRoyy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
excluded_words = ["tempat", "waktu", "mencuri", "rampok"]

NLTK_StopWords = stopwords.words('indonesian')
NLTK_StopWords.extend(["detik", "detikjatim", "detikjateng", "detikjabar", "detiksulsel", "detiksumbar", "detikbali", "detikpapua", "detiksulteng", "detikmaluku", "detjatim", "detikcom", "allahumma", "aamiin", "aamiin", "allah", "bismillah"])
NLTK_StopWords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])
txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
NLTK_StopWords.extend(txt_stopword["stopwords"][0].split(' '))
NLTK_StopWords = set(NLTK_StopWords)

In [3]:
def preprocessing(berita):
    # Preprocessing
    s = berita.lower()
    s = s.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    s = s.encode('ascii', 'replace').decode('ascii')
    ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", s).split())
    s.replace("http://", " ").replace("https://", " ")
    s = re.sub('\s+', ' ', s)
    s = s.strip()
    s = s.translate(str.maketrans("","", string.punctuation))
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    s = re.sub(r'\b\w{1,1}\b', '', s)
    tokens = [token for token in s.split(" ") if token != ""]

    # Stopwords checking
    T = [t for t in tokens if ((t in excluded_words) or (t not in NLTK_StopWords))]
    return T

In [4]:
# df =pd.read_csv("scrapped_news/scrapped_news.csv")
# df.info()
# df.head()
dataframe = []

try:
        cnx = mysql.connector.connect(user = 'root', password='Password', database = 'Petakabar')
        cursor = cnx.cursor()

        ambildata = ("SELECT berita_title, berita_date, berita_desc, berita_source FROM berita where class_classification is not null and berita_topik_id = 5")
        cursor.execute(ambildata)
        dataa = cursor.fetchall()
        for row in dataa:
            dataframe.append(row)
        
except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Something is wrong with your user name or password")
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist")
        else:
            print(err)
else:
    cursor.close()
    cnx.close()

In [6]:
import pandas as pd
writer = pd.DataFrame(dataframe, columns=[
                    'Title','Date','Description', 'Source'], index=None)
writer.to_csv('news.csv', index=False, sep=',')

In [7]:
df =pd.read_csv("news.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2218 entries, 0 to 2217
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        2218 non-null   object
 1   Date         2218 non-null   object
 2   Description  2218 non-null   object
 3   Source       2218 non-null   object
dtypes: object(4)
memory usage: 69.4+ KB


,Title,Date,Description,Source
0,Wali Kota Blitar Ungkap Sumber Uang Rp 400 Jut...,"Rabu, 14 Des 2022 13:20 WIB",Perampok menggondol uang Rp 400 juta dari ruma...,www.tribunnews.com
1,Saat Mobil Perampok Masuk ke Rumah Dinas Wali ...,"Rabu, 14 Des 2022 12:47 WIB",Seorang pria membukakan pintu gerbang utama Ru...,www.tribunnews.com
2,UPDATE Perampokan di Rumah Wali Kota Blitar: P...,"Kamis, 15 Des 2022 16:38 WIB",\- Kamera CCTV merekam sebuah mobil berplat me...,www.tribunnews.com
3,Terungkap Asal Uang Rp 400 Juta yang Dirampok ...,"Rabu, 14 Des 2022 14:41 WIB","Wali Kota Blitar, Jawa Timur, Santoso menjelas...",www.tribunnews.com
4,"Wali Kota Blitar Terpaksa Tunjukkan Brankas, P...","Rabu, 14 Des 2022 08:22 WIB","Wali Kota Blitar, Santoso mengungkapkan detik-...",www.tribunnews.com


In [8]:
column = ['Title', 'Date', 'Description', 'Source']
df_total = pd.DataFrame()

# df = pd.read_csv("scrapped_news/scrapped_news.csv")
df = pd.read_csv("news.csv")
df = df.dropna(subset=['Description'], axis=0)
df.sort_values("Description", inplace = True)
df.drop_duplicates(subset ="Description", keep = False, inplace = True)

df.head()
df_train_unprocessed, df_test_unprocessed = train_test_split(df,test_size=0.1)

In [9]:
for x in range(0, df.shape[0]):
    text = preprocessing(df.iloc[x, -2])
    df.iloc[x, -2] = (' '.join(text)) 

In [10]:
df.head()
df_train, df_test = train_test_split(df,test_size=0.1)
df_train.head()

,Title,Date,Description,Source
251,Bharada E Sempat Cerita Momen Dinas Bareng Bri...,"Rabu, 14 Des 2022 10:23 WIB",kekasih richard eliezer bharada angelin krista...,www.kompas.com
1905,Akhir Pelarian Pembobol Rumah Berakhir Saat Da...,"Jumat, 02 Des 2022 07:37 WIB",polisi perjalanan komplotan spesialis pembobol...,www.detik.com
397,BERITA FOTO: JPU Hadirkan 5 Orang Saksi Ahli d...,"Senin, 19 Des 2022 11:58 WIB",sidang pembunuhan berencana nofriansyah yosua ...,www.kompas.com
51,Pria Dirampok Usai Borong 300 iPhone 13,"Kamis, 01 Des 2022 14:00 WIB",pria 27 memborong 300 iphone 13 dirampok new y...,www.kompas.com
1625,Tampang Pria Penganiaya Anak Kekasih hingga Te...,"Selasa, 06 Des 2022 17:48 WIB",polisi menangkap 31 pelaku pembunuhan balita b...,www.detik.com


In [11]:
desc_text_test_unprocessed = []

for x in range(0, df_test_unprocessed.shape[0]):
  desc_text_test_unprocessed.append(df_test_unprocessed.iloc[x, -2])

joblib.dump(desc_text_test_unprocessed, "corpus/model/desc_text_test_unprocessed.pkl")

['corpus/model/desc_text_test_unprocessed.pkl']

In [12]:
desc_text_train = []
desc_text_test = []

for x in range(0, df_train.shape[0]):
  desc_text_train.append(df_train.iloc[x, -2])

for x in range(0, df_test.shape[0]):
  desc_text_test.append(df_test.iloc[x, -2])

In [13]:
# print(df_test.info())
print("Panjang Dokumen Test : ", len(desc_text_test))

print ('-'*90)

# print(df_train.info())
print("Panjang Dokumen Train : ", len(desc_text_train))

Panjang Dokumen Test :  222
------------------------------------------------------------------------------------------
Panjang Dokumen Train :  1996


In [14]:
# Simpan teks asli dokumen
joblib.dump(desc_text_test, "corpus/model/desc_text_test_new.pkl")
joblib.dump(desc_text_train, "corpus/model/desc_text_train_new.pkl")

# Simpan dataframe hasil text processing sebelum TF-IDF Vectorizer
df.to_csv('corpus/dataset/df_total_new.csv', index=False)
df_test.to_csv('corpus/dataset/df_test.csv', index=False)
df_train.to_csv('corpus/dataset/df_train.csv', index=False)


In [15]:
# Fit transform menggunakan TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_test_matrix = tfidf_vectorizer.fit_transform(desc_text_test)
tfidf_train_matrix = tfidf_vectorizer.fit_transform(desc_text_train)

print('tfidf_test_matrix')
print(tfidf_test_matrix.shape)
print('tfidf_train_matrix')
print(tfidf_train_matrix.shape)

tfidf_test_matrix
(222, 6379)
tfidf_train_matrix
(1996, 17537)


In [16]:
joblib.dump(tfidf_vectorizer, "corpus/vectorizer/vectorizer_new.pkl")
joblib.dump(tfidf_test_matrix, "corpus/matrix/tfidf_test.pkl")
joblib.dump(tfidf_train_matrix, "corpus/matrix/tfidf_train_new.pkl")

['corpus/matrix/tfidf_train_new.pkl']

### Pencarian 

In [19]:
df_total = pd.read_csv('corpus/dataset/df_total_new.csv')
df_total = df_total[pd.notnull(df_total['Description'])]
print(df_total.info())
print ('-'*90)

document_text_train= joblib.load('corpus/model/desc_text_train_new.pkl')
document_text_test= joblib.load('corpus/model/desc_text_test_new.pkl')
print(len(document_text_test))
print(len(document_text_train))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2218 entries, 0 to 2217
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        2218 non-null   object
 1   Date         2218 non-null   object
 2   Description  2218 non-null   object
 3   Source       2218 non-null   object
dtypes: object(4)
memory usage: 86.6+ KB
None
------------------------------------------------------------------------------------------
222
1996


In [20]:
kueri='curi'
kueri=preprocessing(kueri)
kueri= [" ".join (kueri)]
print (kueri)

tfidf_matrix =joblib.load( "corpus/matrix/tfidf_train.pkl" )
tfidf_vectorizer = joblib.load( "corpus/vectorizer/vectorizer.pkl" ) 
query_vec= tfidf_vectorizer.transform(kueri)
results=cosine_similarity(tfidf_matrix, query_vec).reshape((-1))
#print (results)
print("\n======================\n")
print("Top 10 most similar documents in corpus:")

j = 1

for i in results.argsort()[-10:][::-1]:
    print("No ID Dokumen  : ", i)
    print("Tanggal        : ", df_total.iloc[i,1])
    print("Isi berita     : ", df_total.iloc[i,2])
    print("(Score: %.4f) " % results[i])

['curi']


Top 10 most similar documents in corpus:
No ID Dokumen  :  538
Tanggal        :  Selasa, 27 Des 2022 13:02 WIB
Isi berita     :  gadis berusia 14 kabupaten kutai timur kutim kalimantan timur kaltim tewas diperkosa kakeknya pa 38 tubuh korban dibuang busana kolam ditemukan warga kasat reskrim polres kutim iptu made jata wiranegara pelaku pa menenggak miras jumat 2312 malam pelaku mengembalikan sepeda motor pinjam ketemulah korban dirayu ngembaliin motor iptu jata selasa 27122022 melintasi wilayah semaksemak pelaku pengaruh miras menepikan kendaraannya korban curiga memberontak pelaku menangkapnya berontak melawan dipaksa tindak pidana jata korban kuasa melawan tubuh kekar pelaku melawan berteriak teriakan korban pelaku panik kalap menghabisi nyawa korban tersangka panik dicekiklah meninggal jata jata jasad korban dibuang danau terhubung kolam jenazah korban ditemukan kolam warga sabtu 2412 danau nyambung kolam subuh mengapung mayat ditemukan warga telepon polsek dilaporkan ja

c:\Python\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.24.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Python\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.24.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
